## Convert the Ultra Pbp Positions to a more useable format for regression.

### Functions

In [19]:
from bs4 import BeautifulSoup
import requests
from HTMLParser import HTMLParser
from datetime import datetime
import hashlib
import requests
import re

import numpy as np
import pandas as pd

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


# In[58]:
from selenium import webdriver
import urllib2
import MySQLdb

# MAIN


### Read the files

In [20]:
positions_DS = pd.read_csv("Pbp Data Positions.csv", sep=",")
positions_DS.head()

,id,p1pG,p2pG,p3pG,p4pG,p5pG,p1pO,p2pO,p3pO,p4pO,p5pO,secs,score_GSW,score_OPP,sO-sG,game
0,4637,SF,PF,C,SG,PG,SF,PF,PF,SG,PG,302,8,8,0,20160619/CLEGSW
1,4638,SF,PF,SF,SG,PG,SF,PF,PF,SG,PG,84,3,2,-1,20160619/CLEGSW
2,4639,SF,PF,SF,SG,PG,SF,SF,PF,SG,PG,185,5,6,1,20160619/CLEGSW
3,4640,C,PF,SF,SG,PG,SF,SF,PF,SG,PG,0,0,0,0,20160619/CLEGSW
4,4641,C,PF,SF,PG,PG,SF,SF,PF,SG,PG,0,0,0,0,20160619/CLEGSW


In [21]:
positions = []
dataset = pd.DataFrame(columns=('id','SFnum_G', 'PFnum_G', 'Cnum_G', 'SGnum_G', 'PGnum_G', 'SFnum_O', 'PFnum_O', 'Cnum_O', 'SGnum_O', 'PGnum_O', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
print str(datetime.now())
for i,row in positions_DS.iterrows():
    dict_pos_GSW = {'SF':0, 'PF':0, 'C':0, 'SG':0, 'PG':0}
    dict_pos_OPP = {'SF':0, 'PF':0, 'C':0, 'SG':0, 'PG':0}

    
    id_val = []
    end_values = []
    #iterate per row
    for j in range(0,16):
        if j == 0:
            id_val.append(int(row[j]))
        elif j >= 1 and j <= 5:
            dict_pos_GSW[row[j]] += 1
        elif j >= 6 and j <= 10:
            dict_pos_OPP[row[j]] += 1
        elif j != 15:
            end_values.append(int(row[j]))
        else:
            end_values.append(row[j])
    #record the
    record = id_val + [dict_pos_GSW['SF'], dict_pos_GSW['PF'], dict_pos_GSW['C'], dict_pos_GSW['PG'], dict_pos_GSW['SG']]
    record = record + [dict_pos_OPP['SF'], dict_pos_OPP['PF'],dict_pos_OPP['C'],dict_pos_OPP['PG'],dict_pos_OPP['SG']] + end_values
    #print record

    temp = pd.DataFrame([record], columns=('id','SFnum_G', 'PFnum_G', 'Cnum_G', 'SGnum_G', 'PGnum_G', 'SFnum_O', 'PFnum_O', 'Cnum_O', 'SGnum_O', 'PGnum_O', 'secs', 'score_GSW', 'score_OPP', 'sO-sG','game'))
    frames = [dataset, temp]
    dataset = pd.concat(frames, ignore_index = True)

print str(datetime.now())


2018-03-08 15:42:17.899000
2018-03-08 15:42:34.419000


In [22]:
dataset.to_csv("Pbp Positions for Regression.csv", sep=',',index=False)

In [23]:
dataset.head(51)

,id,SFnum_G,PFnum_G,Cnum_G,SGnum_G,PGnum_G,SFnum_O,PFnum_O,Cnum_O,SGnum_O,PGnum_O,secs,score_GSW,score_OPP,sO-sG,game
0,4637,1,1,1,1,1,1,2,0,1,1,302,8,8,0,20160619/CLEGSW
1,4638,2,1,0,1,1,1,2,0,1,1,84,3,2,-1,20160619/CLEGSW
2,4639,2,1,0,1,1,2,1,0,1,1,185,5,6,1,20160619/CLEGSW
3,4640,1,1,1,1,1,2,1,0,1,1,0,0,0,0,20160619/CLEGSW
4,4641,1,1,1,2,0,2,1,0,1,1,0,0,0,0,20160619/CLEGSW
5,4642,1,1,1,2,0,2,1,0,1,1,53,3,2,-1,20160619/CLEGSW
6,4643,1,1,1,2,0,1,1,0,1,2,30,2,1,-1,20160619/CLEGSW
7,4644,2,0,1,2,0,1,1,0,1,2,41,1,2,1,20160619/CLEGSW
8,4645,2,0,1,1,1,1,1,0,1,2,0,0,0,0,20160619/CLEGSW
9,4646,2,0,1,1,1,2,1,0,1,1,21,0,1,1,20160619/CLEGSW


## After this, we use the "Pbp Positions for Regression.csv" and the "NBA Dataset with RPM.csv" to help us do regression and figure out which players from the opponent's side has the best chances of defeating the GSW.